In [1]:
!pip install ultralytics tensorflow opencv-python-headless

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# Create directories
os.makedirs("/content/synthetic_data/L_shape", exist_ok=True)
os.makedirs("/content/synthetic_data/Wrong_path", exist_ok=True)

# Function to generate L-shape frames (red square)
def generate_l_shape_frames():
    width, height = 640, 480
    vehicle_size = 20
    frames = []
    x, y = 500, 400
    for i in range(20):
        frame = np.ones((height, width, 3), dtype=np.uint8) * 255
        cv2.rectangle(frame, (x, y), (x+vehicle_size, y+vehicle_size), (0,0,255), -1)
        frames.append(frame)
        x -= 10  # move left
    for i in range(20):
        frame = np.ones((height, width, 3), dtype=np.uint8) * 255
        cv2.rectangle(frame, (x, y), (x+vehicle_size, y+vehicle_size), (0,0,255), -1)
        frames.append(frame)
        y -= 10  # move up
    return frames

# Function to generate Wrong path frames (blue square)
def generate_wrong_path_frames():
    width, height = 640, 480
    vehicle_size = 20
    frames = []
    x, y = 500, 400
    for i in range(10):
        frame = np.ones((height, width, 3), dtype=np.uint8) * 255
        cv2.rectangle(frame, (x, y), (x+vehicle_size, y+vehicle_size), (255,0,0), -1)
        frames.append(frame)
        x -= 10
        if i % 2 == 0:
            y -= 10
    return frames

# Save frames as images
def save_frames(frames, folder):
    for i, frame in enumerate(frames):
        cv2.imwrite(f"{folder}/frame_{i:03}.png", frame)

# Generate and save datasets
l_frames = generate_l_shape_frames()
wrong_frames = generate_wrong_path_frames()

save_frames(l_frames, "/content/synthetic_data/L_shape")
save_frames(wrong_frames, "/content/synthetic_data/Wrong_path")


In [3]:
def create_video_from_frames(folder, output_path):
    images = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, _ = frame.shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))
    for img_name in images:
        img = cv2.imread(os.path.join(folder, img_name))
        out.write(img)
    out.release()

create_video_from_frames("/content/synthetic_data/L_shape", "/content/L_shape_video.mp4")
create_video_from_frames("/content/synthetic_data/Wrong_path", "/content/Wrong_path_video.mp4")


In [4]:
# Load pretrained YOLOv8 model
model = YOLO("yolov8n.pt")

def detect_vehicle_centers(video_path):
    cap = cv2.VideoCapture(video_path)
    centers = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        boxes = results[0].boxes
        centers_frame = []
        for box in boxes:
            cls_id = int(box.cls[0])
            cls_name = model.names[cls_id]
            if cls_name in ["car", "truck", "bus"]:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2
                centers_frame.append((cx, cy))
        # Pick the first detected vehicle center if any
        centers.append(centers_frame[0] if centers_frame else None)
    cap.release()
    return centers

# Example:
l_centers = detect_vehicle_centers("/content/L_shape_video.mp4")
wrong_centers = detect_vehicle_centers("/content/Wrong_path_video.mp4")

print("L shape centers sample:", l_centers[:5])
print("Wrong path centers sample:", wrong_centers[:5])


100%|██████████| 6.25M/6.25M [00:00<00:00, 53.6MB/s]



0: 480x640 (no detections), 309.7ms
Speed: 5.4ms preprocess, 309.7ms inference, 9.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 157.1ms
Speed: 3.1ms preprocess, 157.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 170.3ms
Speed: 3.4ms preprocess, 170.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 153.7ms
Speed: 4.4ms preprocess, 153.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 154.3ms
Speed: 4.2ms preprocess, 154.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 156.6ms
Speed: 4.4ms preprocess, 156.6ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 219.6ms
Speed: 3.0ms preprocess, 219.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 258.3ms
Speed: 2.7ms prepr

In [5]:
# Assume l_centers and wrong_centers are already detected

def preprocess_centers(centers, seq_length=30):
    processed = []
    prev = (0, 0)
    for c in centers:
        if c is None:
            processed.append(prev)
        else:
            processed.append(c)
            prev = c
    if len(processed) < seq_length:
        processed = processed + [processed[-1]] * (seq_length - len(processed))
    else:
        processed = processed[:seq_length]
    return np.array(processed)

seq_len = 30  # define globally so it's accessible

X = []
y = []

X.append(preprocess_centers(l_centers, seq_len))
y.append(1)  # Label: L-shape = 1

X.append(preprocess_centers(wrong_centers, seq_len))
y.append(0)  # Label: wrong path = 0

X = np.array(X)
y = np.array(y)


In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, RNN, LSTMCell

seq_len = 30

model = Sequential([
    Input(shape=(seq_len, 2)),
    RNN(LSTMCell(64), return_sequences=True),
    RNN(LSTMCell(32)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X, y, epochs=30, batch_size=1)


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.0000e+00 - loss: 0.6933
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6667 - loss: 0.6930
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3333 - loss: 0.6934    
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3333 - loss: 0.6933    
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6667 - loss: 0.6931
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3333 - loss: 0.6933    
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6667 - loss: 0.6930
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6667 - loss: 0.6930
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6667 - loss: 0.6930
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6667 - loss: 0.6930
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3333 - loss: 0.6934    
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6

In [10]:
# If using a Keras Sequential/Functional model, do this:
model.export('/content/l_shape_model')  # Not .save()


Saved artifact at '/content/l_shape_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136897345406160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897384709776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345406736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345408464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345408656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345405968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345406928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345410384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345409424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136897345411152: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [11]:
!pip install tensorflowjs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [1]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --signature_name=serving_default \
    --output_format=tfjs_graph_model \
    /content/l_shape_model \
    /content/tfjs_model


2025-07-07 19:08:31.689787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751915311.721567    5665 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751915311.730509    5665 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-07 19:08:31.766468: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-0

In [2]:
!zip -r tfjs_model.zip tfjs_model


  adding: tfjs_model/ (stored 0%)
  adding: tfjs_model/group1-shard1of1.bin (deflated 9%)
  adding: tfjs_model/model.json (deflated 94%)
